# Credit Card Data Analysis for Customer Approval

##### Description of data columns

* card: Dummy variable, 1 if application for credit card accepted, 0 if not
* reports: Number of major derogatory reports
* age: Age n years plus twelfths of a year
* income: Yearly income (divided by 10,000)
* share: Ratio of monthly credit card expenditure to yearly income
* expenditure: Average monthly credit card expenditure
* owner: 1 if owns their home, 0 if rent
* selfempl: 1 if self employed, 0 if not.
* dependents: 1 + number of dependents
* months: Months living at current address
* majorcards: Number of major credit cards held
* active: Number of active credit accounts

### Ask Phase



1. What type of company does your client represent, and what are they asking you to accomplish?
    - The client represent a bank, based on the credit card data bank needs to make decision on which customers would benifit from the credit card and which customers would be unable to pay back the money. based on the previous customer data bank need to make decision on upcoming customers to approve credit cards in such a way that it won't be a burden to the company.

2. What are the key factors involved in the business task you are investigating?
    - For the current scenario client has provided a dataset of credit card application data, which contains card approved or not, age, number of derogatory reports, ownership of house, etc. by analysing this dataset we need pull out insights about the card holders.
        Below are the key factors that under investigation:
        - Who are the customers maintaining credit card account without failing the repayment
        - Who are the customers failing to repay the amount
    Based on these two factors more sub queries has to be made, asking underlying reason for the outcome would make a clear picture.

3. What type of data will be appropriate for your analysis?
    - client has provided credit card application dataset which would be used for this analysis.

4. Where will you obtain that data?
    - Client provided data

5. Who is your audience, and what materials will help you present to them effectively?
    - We will be working with financial analyst, other stakeholder like financial department manager and CFO will be involved in this project


### Prepare Phase



1. Where is your data located?
    - Google BigQuery
2. How is the data organized?
    - As a table (only one data set is available)
3. Are there issues with bias or credibility in this data? Does your data ROCCC?
    - Yes it does satisfies the ROCCC concepts
4. How are you addressing licensing, privacy, security, and accessibility?
    - Data given by client so all of them are considered by the client
5. How did you verify the data’s integrity?
    - Provided by the client
6. How does it help you answer your question?
    - Dataset contains all required columns so it would be sufficient to answer the questions
7. Are there any problems with the data?
    - Data needs to be cleaned since it contains null values and format of the data

### Process Phase


1. For the processing of data duplicated the table with a name process_data
``` 
CREATE TABLE
  `creditcard_dataset.processed_data` AS (
  SELECT
    *
  FROM
    `creditcard_dataset.creditcard`)
```


2. The age column is in decimal format which is not usually used, it is changed to integer by flooring the age
``` 
UPDATE `creditcard_dataset.processed_data`
SET age = FLOOR(age)
WHERE TRUE
```



3. Few rows contains age = 0 which is not logically correct, so deleting the rows since it doesn't significantly impact the analysis
``` 
DELETE FROM `creditcard_dataset.processed_data`
where age < 18
```


4. The income column is devided by 10,000, we don't need that right now so converting it back to the normal form.
``` 
Update `creditcard_dataset.processed_data`
set income = income * 10000
where TRUE
```


5. changing the decimal points to 2 since we don't need so many trailing zeros after the decimal points
``` 
Update `creditcard_dataset.processed_data`
set income = ROUND(income, 2)
where TRUE
```


6. share column seems to be incorrect where it is the ratio of monthly expenditure devided by anual income
``` 
Update `creditcard_dataset.processed_data`
set share = expenditure/income
where TRUE
```


7. There are many accounts with more than 10 active credit accounts which is suspecious and not practically attainable from a financial perspective, we need to check summarised count of active accounts
``` select active, count(active)
from `creditcard_dataset.processed_data`
group by active

8. Checking the number of major cards with each customer
``` select majorcards, count(majorcards)
from `creditcard_dataset.processed_data`
group by majorcards

### Since all the data is cleaned for analysis we can proceed to the analysis phace, **but we might have to exclude active column due to inconsitency in the data**.

### Analysis Phase

1. Income destribution of customers whose application is accepted
```
select
count(income) as count,
min(income) as minimum,
max(income) as maximum,
avg(income) as mean
FROM
`creditcard_dataset.processed_data`
where
card = true
```

**Query Result:**

count|minimum|maximum|mean
-----|-------|-------|------
1017|2100.0|135000.0|34538.469026548628

2. Income destribution of customers whose application is rejected
``` select
count(income) as count,
min(income) as minimum,
max(income) as maximum,
avg(income) as mean
FROM
`creditcard_dataset.processed_data`
where
card = false
```

**Query Result:**

count|minimum|maximum|mean
-----|-------|-------|-----
295|4900.0|110000.0|30687.413559322034

#### As you can see above data, even though the income ranges are similar customers are rejected. **Hence the income alone doesn't have significant impact on card approval.**

3. Once we look at the age of the approved card holders there is a inverse relationship between age and approval, younger customers are approved more than the elder customers as you can see in the grouped data

 ```
 SELECT CASE
         WHEN age BETWEEN 18 AND 29 THEN '18-29'
         WHEN age BETWEEN 30 AND 39 THEN '30-39'
         WHEN age BETWEEN 40 AND 49 THEN '40-49'
         WHEN age BETWEEN 50 AND 69 THEN '50-69'
         ELSE '70+'
       END AS age_range,
       COUNT(*) AS count
FROM `creditcard_dataset.processed_data`
GROUP BY age_range
ORDER BY age_range;
```

**Query Result:**

| age_range | count |
|-----------|-------|
| 18-29     | 582   |
| 30-39     | 421   |
| 40-49     | 223   |
| 50-69     | 82    |
| 70+       | 4     |



4. As we can see in the query result; in approved customers home owners and non-home owners are almost similar amount. **But when it comes to rejected customers, non-home owners are much higher than home owners. so home ownership is a key factor in card approval**. for those who are non-home owners and still approved, there might be some other factors enhancing there chances of getting a card.

**Approved Customers:**
```
SELECT
  owner,
  COUNT(owner) as Count
FROM
  `creditcard_dataset.processed_data`
WHERE
  card = true
GROUP BY
  owner
```

**Query Result:**

| Owner | Count |
|-------|-------|
| true  | 489   |
| false | 528   |

-----------------------

**Rejected Customers:**
```
SELECT
  owner,
  COUNT(owner) as Count
FROM
  `creditcard_dataset.processed_data`
WHERE
  card = false
GROUP BY
  owner
```

**Query Result:**

| Owner | Count |
|-------|-------|
| true  | 90    |
| false | 205   |


5. The below describes the tenure of the customers lived in their current address. Tenure doesn't affect the card approval much, but it is a key factor in card rejection. **Customers who have lived in their current address for less than 10 years are more likely to get rejected.**

***Card Application Rejected Customers:***

```
SELECT
CASE
  WHEN months <= 12 THEN '<= 1 Year'
  WHEN months <= 24 THEN '<= 2 Years'
  WHEN months <= 36 THEN '<= 3 Years'
  WHEN months <= 48 THEN '<= 4 Years'
  WHEN months <= 60 THEN '<= 5 Years'
  WHEN months <= 72 THEN '<= 6 Years'
  WHEN months <= 84 THEN '<= 7 Years'
  WHEN months <= 96 THEN '<= 8 Years'
  WHEN months <= 108 THEN '<= 9 Years'
  WHEN months <= 120 THEN '<= 10 Years'
  WHEN months <= 132 THEN '<= 11 Years'
  WHEN months <= 144 THEN '<= 12 Years'
  WHEN months <= 156 THEN '<= 13 Years'
  WHEN months <= 168 THEN '<= 14 Years'
  WHEN months <= 180 THEN '<= 15 Years'
  WHEN months <= 192 THEN '<= 16 Years'
  WHEN months <= 204 THEN '<= 17 Years'
  WHEN months <= 216 THEN '<= 18 Years'
  WHEN months <= 228 THEN '<= 19 Years'
  ELSE '<= 20 Years'
END Tenure,
  count(*) as Count
FROM
  `creditcard_dataset.processed_data`
WHERE
  card = false
group by Tenure
order by Count

```
**Query Result:**

| Tenure | Count |
|---|---|
| <= 16 Years | 1 |
| <= 14 Years | 1 |
| <= 17 Years | 1 |
| <= 18 Years | 1 |
| <= 19 Years | 2 |
| <= 15 Years | 2 |
| <= 13 Years | 2 |
| <= 12 Years | 3 |
| <= 11 Years | 3 |
| <= 10 Years | 4 |
| <= 8 Years | 9 |
| <= 20 Years | 10 |
| <= 7 Years | 12 |
| <= 9 Years | 12 |
| <= 6 Years | 17 |
| <= 5 Years | 24 |
| <= 4 Years | 28 |
| <= 3 Years | 49 |
| <= 2 Years | 56 |
| <= 1 Year | 58 |

------------------------
***Card Application Approved Customers:***

```
SELECT
CASE
  WHEN months <= 12 THEN '<= 1 Year'
  WHEN months <= 24 THEN '<= 2 Years'
  WHEN months <= 36 THEN '<= 3 Years'
  WHEN months <= 48 THEN '<= 4 Years'
  WHEN months <= 60 THEN '<= 5 Years'
  WHEN months <= 72 THEN '<= 6 Years'
  WHEN months <= 84 THEN '<= 7 Years'
  WHEN months <= 96 THEN '<= 8 Years'
  WHEN months <= 108 THEN '<= 9 Years'
  WHEN months <= 120 THEN '<= 10 Years'
  WHEN months <= 132 THEN '<= 11 Years'
  WHEN months <= 144 THEN '<= 12 Years'
  WHEN months <= 156 THEN '<= 13 Years'
  WHEN months <= 168 THEN '<= 14 Years'
  WHEN months <= 180 THEN '<= 15 Years'
  WHEN months <= 192 THEN '<= 16 Years'
  WHEN months <= 204 THEN '<= 17 Years'
  WHEN months <= 216 THEN '<= 18 Years'
  WHEN months <= 228 THEN '<= 19 Years'
  ELSE '<= 20 Years'
END Tenure,
  count(*) as Count
FROM
  `creditcard_dataset.processed_data`
WHERE
  card = true
group by Tenure
order by Count

```

**Query Result:**

| Tenure | Count |
|---|---|
| <= 19 Years | 5 |
| <= 17 Years | 5 |
| <= 18 Years | 6 |
| <= 16 Years | 9 |
| <= 15 Years | 11 |
| <= 14 Years | 15 |
| <= 12 Years | 17 |
| <= 13 Years | 18 |
| <= 9 Years | 20 |
| <= 11 Years | 21 |
| <= 7 Years | 28 |
| <= 8 Years | 29 |
| <= 20 Years | 31 |
| <= 10 Years | 34 |
| <= 6 Years | 39 |
| <= 5 Years | 72 |
| <= 4 Years | 80 |
| <= 3 Years | 116 |
| <= 2 Years | 179 |
| <= 1 Year | 282 |

**Improved Query:**

```
select
ceil(months/12) as tenure,
count(card = true or null) as approved,
count(card = false or null) as rejected
from
`creditcard_dataset.processed_data`
group by tenure
order by tenure
```

**Query Result:**

|tenure|approved|rejected|
|------|--------|--------|
|0.0   |1       |2       |
|1.0   |281     |56      |
|2.0   |179     |56      |
|3.0   |116     |49      |
|4.0   |80      |28      |
|5.0   |72      |24      |
|6.0   |39      |17      |
|7.0   |28      |12      |
|8.0   |29      |9       |
|9.0   |20      |12      |
|10.0  |34      |4       |
|11.0  |21      |3       |
|12.0  |17      |3       |
|13.0  |18      |2       |
|14.0  |15      |1       |
|15.0  |11      |2       |
|16.0  |9       |1       |
|17.0  |5       |1       |
|18.0  |6       |1       |
|19.0  |5       |2       |
|20.0  |9       |1       |
|21.0  |7       |1       |
|22.0  |1       |0       |
|23.0  |4       |0       |
|24.0  |2       |1       |
|25.0  |3       |1       |
|26.0  |2       |0       |
|29.0  |0       |1       |
|31.0  |1       |1       |
|34.0  |0       |2       |
|36.0  |0       |1       |
|43.0  |1       |0       |
|44.0  |0       |1       |
|45.0  |1       |0       |


6. Customers who is self-employed are very less compared to others. Self-employed customers are more likely to get rejected than other employed or business customers. **Employed/Business customers are more likely to get approved than self-employed customers.**

**Customer who is not self-employed:**

```
SELECT
  card, Count(*) as selfemp_count
FROM
  `creditcard_dataset.processed_data`
WHERE
  selfemp = False
group by card
```

**Query Result:**

| card | selfemp_count |
|---|---|
| false | 267 |
| true | 954 |

-----------------------

**Customer who is self-employed:**

```
SELECT
  card, Count(*) as selfemp_count
FROM
  `creditcard_dataset.processed_data`
WHERE
  selfemp = True
group by card
```

**Query Result:**

| card | selfemp_count |
|---|---|
| false | 28 |
| true | 63 |


7. Derogatory reports are the negative reports about the customer. Customers with more derogatory reports are more likely to get rejected. Customers with less derogatory reports are more likely to get approved. but customers with 0 derogatory are also rejected by the bank this could be because of other factors.

**Card Application Rejected Customers:**

```
select
reports, count(reports) as derogatory_reports
from
`creditcard_dataset.processed_data`
where card = false
group by reports
order by derogatory_reports
```

**Query Result:**

| reports | derogatory_reports |
|---|---|
| 10 | 1 |
| 12 | 1 |
| 14 | 1 |
| 9 | 2 |
| 11 | 4 |
| 6 | 5 |
| 7 | 6 |
| 5 | 11 |
| 4 | 16 |
| 3 | 20 |
| 2 | 37 |
| 1 | 47 |
| 0 | 144 |

------------------------

**Card Application Approved Customers:**

```
select
reports, count(reports) as derogatory_reports
from
`creditcard_dataset.processed_data`
where card = true
group by reports
order by derogatory_reports
```

**Query Result:**

| reports | derogatory_reports |
|---|---|
| 4 | 1 |
| 3 | 4 |
| 2 | 13 |
| 1 | 89 |
| 0 | 910 |



### Share Phase

#### Tableau Visualisation - [Link to the Dashboard](https://public.tableau.com/views/FinancialInsight/Dashboard1?:language=en-GB&:display_count=n&:origin=viz_share_link)